In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

## 구조 만들기

In [3]:
'''
Transformer(num_layers, input_src, input_tgt, max_vocab, seq_length, dim_embedding, num_head, dff)
    Encoder(num_layers, input_src, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)
        Encoder layer(input_src, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)
            vectorization(input_src, max_vacab, seq_length)
            embedding(input_src2, max_vacab, dim_embedding)
            positional encoding(input_src3, position=seq_length, depth=dim_embedding)
            multi-head self attention(input_src4, dim_model=dim_embedding, num_head)
                pad_mask
            add&normalization(input_src5)
            feed forward neural network(input_src6, dim_model, dff)
            add&normalization(input_src7)
    Decoder(num_layers, num_layers, input, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)
        Decoder layer(input_tgt, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff, enc_out)
            vectorization(input_tgt, max_vacab, seq_length)
            embedding(input_tgt2, max_vacab, dim_embedding)
            positional encoding(input_tgt3, position=seq_length, depth=dim_embedding)
            masked multi-head self attention(input_tgt4, dim_model=dim_embedding, num_head)
                mask, pad_mask
            add&normalization(input_tgt5)
            cross multi-head self attention(input_tgt6, enc_output, pad_mask, dim_model=dim_embedding, num_head)
            add&normalization(input_tgt7)
            feed forward neural network(input_tgt8, dim_model, dff)
            add&normalization(input_tgt9)
    Generator
        Linear(dec_out)
        Softmax(linear_out)
            
'''

'\nTransformer(num_layers, input_src, input_tgt, max_vocab, seq_length, dim_embedding, num_head, dff)\n    Encoder(num_layers, input_src, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)\n        Encoder layer(input_src, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)\n            vectorization(input_src, max_vacab, seq_length)\n            embedding(input_src2, max_vacab, dim_embedding)\n            positional encoding(input_src3, position=seq_length, depth=dim_embedding)\n            multi-head self attention(input_src4, dim_model=dim_embedding, num_head)\n                pad_mask\n            add&normalization(input_src5)\n            feed forward neural network(input_src6, dim_model, dff)\n            add&normalization(input_src7)\n    Decoder(num_layers, num_layers, input, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff)\n        Decoder layer(input_tgt, max_vacab, seq_length, dim_embedding, pad_mask, num_head, dff, enc_out)\n            

## Positional Encoding

In [4]:
def positional_encoding(inputs):
    #input의 형태는 문장을 vectorization -> embedding한 (batch_size, seq_length, dim_embedding)의 형태로 입력이 됨
    #batch_size는 모델의 파라미터로 한번에 처리할 문장의 갯수
    #seq_length는 한 문장에서 처리할 Token의 숫자
    #dim_embedding은 한개의 token을 몇 차원으로 embedding하는지 나타내는 값
    seq_length = inputs.shape[-2]
    dim_embedding = inputs.shape[-1]
    positional_encoding = np.zeros((seq_length, dim_embedding))

    
    #(seq_length, dim_embedding)의 구조로 된 문장을 positional encoding을 하기 위해 pos(행), i(열) 좌표(?) 위치를 계산
    pos = np.arange(seq_length)[:,np.newaxis] #행의 번호
    i = np.arange(dim_embedding)[np.newaxis,:] #열의 번호
    d_model = dim_embedding

    #positional encoding angle 공식
    angle = pos*1/np.power(10000,(2 * (i // 2) / np.float32(dim_embedding)))

    #i가 짝수 일때는 sin, i가 홀수 일때는 cos으로 계산하여, 위치별로 더해줄 positional encoding 값을 계산
    positional_encoding[:, 0::2] = np.sin(angle[:, 0::2])
    positional_encoding[:, 1::2] = np.cos(angle[:, 1::2])
    positional_encoding[tf.newaxis, ...] #(batch_size, seq_length, dim_embedding)형태로 만들어 주기 위해 batch_size 부분에 해당하는 np.newaxis 추가

    return tf.cast(positional_encoding, dtype=tf.float32) #output 차원 = (batch_size, seq_length, dim_embedding)

## scaled_dot_product & split_heads

In [5]:
#self attention에서는 input으로 embedded positional encoded sentence를 받고 query, key, value를 만들어 self attention score를 얻어낸다.
#필요한 input: input

#input을 받아서 query, key, value를 만든다.
#matmul : matmul(query,key.transpose)
#scaling : matmul/tf.sqrt(dim_k)
#pad_mask : scaled + pad_mask
#softmax : softmax(pad_masked)
#output : matmul(softmax, value)

def create_pad_mask(x):
    #여기서 input 값인 x는 vectorize된 문장이 들어가야 함
    #input 형태 : (batch_size, seq_length) -> [[1,2,3,1,0,0,0],[3,2,4,0,0,0,0]] 이런 형태로 입력
    #matirx에서 x값이 0과 같은 경우, True, 아니면 False를 반환하고, tf.cast를 통해서 boolen을 float32로 변환(True = 1.0, False=0.0)
    mask = tf.cast(tf.math.equal(x, 0), dtype=tf.float32)
    #return은 multi-head self attention에서 사용하기 위해 (batch_size, num_head, seq_length, seq_length)의 형태로 만들어줌
    #batch_size는 문장의 숫자가 되고, num_head는 multi-head를 사용할 때 필요한 차원이고, scaled_dot_product에서 mask를 적용해야 할 matmul_qk의 차원이 seq_length, seq_length인데 우리는 Column에다가 mask를 적용할 것이기 떄문에, (Batch_size, 1, 1, seq_length)의 차원으로 만들어줌
    return mask[:, tf.newaxis, tf.newaxis, :] #output 차원 (batch_size, 1, 1, seq_length)


def scaled_dot_product(query, key, value, pad_mask=None):
    # query와 key의 transpose dot product이후 scaling
    # 입력 query, key, value의 차원 = (batch_size, seq_length, dim_k)
    # 즉 입력은 Embedding layer와 Positional encoding layer를 통과한 후의 데이터가 입력 되면 됨
    matmul_qk = tf.matmul(query, key, transpose_b = True) #차원 = (batch_szie, seq_length, seq_length)
    dim_k = tf.cast(key.shape[-1], tf.float32)
    scaled_matmul_qk = matmul_qk / tf.math.sqrt(tf.cast(dim_k, tf.float32)) #차원 = (batch_size, seq_length, seq_length)

    if pad_mask is not None:
        # 패딩된 위치에 -1e9를 더해 softmax에서 0에 가까운 값이 되도록 함 
        # pad mask의 차원은 (batch_size, num_head, seq_length, seq_length)이므로, multi_head self attention에서 사용해야함
        scaled_matmul_qk += (pad_mask * -1e9) #차원은 (batch_size, (num_head), seq_length, seq_length)
    
    # 소프트맥스 적용
    softmax_qk = tf.nn.softmax(scaled_matmul_qk, axis=-1) #행 방향으로 softmax를 하고, 차원은 변경 없이 (batch_size, (num_head), seq_length, seq_length)
    
    # Attention 결과 계산
    attention = tf.matmul(softmax_qk, value) #차원은 (batch_size, (num_head), seq_length, dim_k)
    
    return attention

In [6]:
def split_heads(x, batch_size, num_head, depth):
    #multi-head self attention을 하기 원래 query, key, value(batch_size, seq_length, dim_k)를 num_haed만큼 concat을 시켜서 최종 attention을 얻어야 한다.
    #이를 병렬 처리 하기 위해서 query, key, value를 (batch_size, seq_length, num_head, depth(뒤에서는 dim_k=dim_model/num_head로 표현))의 형태로 한번에 변환을 해줌
    #reshape과정에서 데이터가 바뀌면 안되기 때문에, 원래의 query (batch_size, seq_length, dim_model)에서 dim_model을 num_head로 잘라내고 남은 depth를 마지막 차원에 둠, -1을 넣는 것은 seq_length 자리임, 자동 계산되고 남은 숫자가 seq_length에 들어가게 됨
    #마지막으로 병렬처리르 할 수 있게 (batch, seq_length, num_head, depth)인 데이터를 (batch_size, num_head, seq_length, depth)로 변경해서, num_head만큼 동시에 병렬처리를 진행
    x = tf.reshape(x, (batch_size, -1, num_head, depth))
    return tf.transpose(x, perm=[0,2,1,3])

## multi-head self attention

In [7]:
class Multiheadselfattention():
    def __init__(self, dim_model, num_head):
        self.dim_model = dim_model
        self.num_head = num_head
        self.depth = dim_model // num_head

        # query, key, value에 Dense layer를 생성
        self.q_layer = tf.keras.layers.Dense(dim_model)
        self.k_layer = tf.keras.layers.Dense(dim_model)
        self.v_layer = tf.keras.layers.Dense(dim_model)

        # 최종 계산된 attention output을 입력값 차원과 동일하게 dim_embedding으로 바꿔주기 위해서 Dense_layer 사용
        self.dim_embedding = dim_model
        self.dense_layer = tf.keras.layers.Dense(self.dim_embedding)
            
    def __call__(self, query, key, value, pad_mask):
        #batch_size를 입력된 data를 통해서 확인
        batch_size = tf.shape(query)[0]

        #위에서 만든 각각의 Dense layer를 이용하여 query, key, value 계산
        q = self.q_layer(query)
        k = self.k_layer(key)
        v = self.v_layer(value)

        #한번에 계산이된 dim_model 차원의 query, key, value 값을 위에서 정의한 split_head 함수로 num_head개의 depth 차원으로 분할
        q = split_heads(q, batch_size, self.num_head, self.depth)
        k = split_heads(k, batch_size, self.num_head, self.depth)
        v = split_heads(v, batch_size, self.num_head, self.depth)

        #위에서 정의한 scaled_dot_product에 q, k, v, 그리고 input data에 맞게 생성된 pad_mask를 입력하여 self attention 연산
        attention = scaled_dot_product(q, k, v, pad_mask) #입력된 값은 (batch_size, num_head, seq_length, depth)차원이고, output역시 동일하게 (batch_size, num_head, seq_length, depth)가 된다.
        attention_scaled = tf.transpose(attention, perm=[0,2,1,3]) #위의 output이 (batch_size, num_head, seq_length, depth)이기 떄문에 다시 concat을 하고 원래 최초에 들어온 input인 (batch_size, seq_length, dim_model) 차원을 맞추기 위해서 (batch_size, seq_length, num_head, depth)로 차원을 바꿔준다.
        attention_concat = tf.reshape(attention_scaled, (batch_size, -1, self.dim_model)) #위에서 바꾼 차원을 값을 num_head와 depth를 합쳐 (batch_size, seq_length, dim_model로 바꿔줌
        attention_output = self.dense_layer(attention_concat)

        return attention_output

In [8]:
def create_embedding_sample(batch_size=32, seq_length=40, embedding_dim=512):
    """
    임베딩된 텍스트를 시뮬레이션하는 샘플 데이터를 생성합니다.
    
    Args:
    - batch_size: 배치 크기
    - seq_length: 시퀀스 길이
    - embedding_dim: 임베딩 차원
    
    Returns:
    - 생성된 샘플 데이터 (shape: [batch_size, seq_length, embedding_dim])
    """
    # 정규 분포를 사용하여 랜덤한 임베딩 생성
    sample_data = tf.random.normal(shape=[batch_size, seq_length, embedding_dim])
    
    return sample_data

# 샘플 데이터 생성
sample_embeddings = create_embedding_sample()

## Fead Forward Neural Network

In [9]:
class Feadforwardneuralnetwork():
    def __init__(self, dim_model, dim_ff):
        self.dim_ff = dim_ff
        self.dim_model = dim_model
        self.dense_layer1 = tf.keras.layers.Dense(self.dim_ff, activation='relu')
        self.dense_layer2 = tf.keras.layers.Dense(self.dim_model)

    def __call__(self, x):
        dense_layer2 = tf.keras.layers.Dense(self.dim_model)
        x1 = self.dense_layer1(x)
        output_ffnn = self.dense_layer2(x1)
        return output_ffnn

## Encoder layer

In [10]:
class Encoderlayer():
    def __init__(self, dim_model, num_head, dim_ff):
        self.dim_model = dim_model
        self.num_head = num_head
        self.dim_ff = dim_ff
        
        self.mha = Multiheadselfattention(self.dim_model, self.num_head)
        self.ffnn = Feadforwardneuralnetwork(self.dim_model, self.dim_ff)

        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
    def __call__(self, x, pad_mask):
        attention = self.mha(x, x, x, pad_mask)

        attention_norm = self.norm1(attention+x)

        output_ffnn = self.ffnn(attention_norm)

        output_ffnn_norm = self.norm2(output_ffnn+attention_norm)

        return output_ffnn_norm

## look ahead mask

In [11]:
def create_lookaheadmask(seq_length):
    #tf.ones((seq_length, seq_length))는 seq_length*seq_length의 matrix에 1을 가득 채운 것임
    #tf.linalg.band_part(matrix, num_lower, num_upper)에서 num_lower는 대각선 아래 유지할 행의 개수, num_upper는 대각선 위 유지할 열의 개수이고, -1을 입력하면 전체를 의미하고, 유지하지 않는 부분은 0으로 채움
    #1-tf.linalg~~~ 를 진행하는 이유는 나중에 1e-9를 곱해주어서 softmax에서 0을 만들기 위함
    mask = 1-tf.linalg.band_part(tf.ones((seq_length, seq_length)), -1, 0)
    return mask

def create_mask(x):
    #padding과 look ahead mask를 한번에 만들어주는 함수
    #input으로 사용되는 x는 vectorized된 문장이다. 
    #input 형태 : (batch_size, seq_length) -> [[1,2,3,1,0,0,0],[3,2,4,0,0,0,0]] 이런 형태로 입력
    temp_mask = create_lookaheadmask(tf.shape(x)[1]) #temp_mask는 위의 create_lookaheadmask를 그대로 실행해서 만듬
    reverse_tar = tf.cast(tf.math.equal(x, 0),dtype=tf.float32) #입력이 되는 tar 문장의 pad mask이다. 즉 pad 부분을 1로, 단어가 있는 부분을 0으로 만드는 식
    reverse_tar = reverse_tar[:,tf.newaxis,tf.newaxis,:] #나중에 (batch_size, num_head, seq_length, seq_length) 형태로 더해주기 위해 형태 변형 -> (batch_size, 1, 1, seq_length)형태
    look_ahead_mask = tf.maximum(reverse_tar, temp_mask) #최종형태는 pad와 look_ahead 부분을 모두 가리는 mask
    return look_ahead_mask

## Decoder layer

In [12]:
class Decoder_layer():
    def __init__(self, dim_model, num_head, dim_ff):
        self.dim_model = dim_model
        self.dim_ff = dim_ff
        self.num_head = num_head

        self.mha1 = Multiheadselfattention(self.dim_model, self.num_head)
        self.mha2 = Multiheadselfattention(self.dim_model, self.num_head)
        self.ffnn = Feadforwardneuralnetwork(self.dim_model, self.dim_ff)
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def __call__(self, tar, out_enc, pad_mask, look_ahead_mask):        
        attention = self.mha1(tar, tar, tar, look_ahead_mask)

        attention_norm = self.norm1(attention+tar)

        cross_attention = self.mha2(attention_norm, out_enc, out_enc, pad_mask)

        cross_attention_norm = self.norm2(cross_attention + attention_norm)

        ffnn = self.ffnn(cross_attention_norm)

        ffnn_norm = self.norm3(ffnn+cross_attention_norm)

        return ffnn_norm       

## Encoder

In [13]:
class Encoder():
    def __init__(self, dim_model, num_head, dim_ff, num_layer):
        self.dim_model = dim_model
        self.num_head = num_head
        self.dim_ff = dim_ff
        self.num_layer = num_layer

        self.encoder_block = [Encoderlayer(self.dim_model,self.num_head,self.dim_ff) for _ in range(self.num_layer)]

    def __call__(self, x, pad_mask):
        for i in range(self.num_layer):
            x = self.encoder_block[i](x, pad_mask)
        return x

## Decoder

In [14]:
class Decoder():
    def __init__(self, dim_model, num_head, dim_ff, num_layer):
        self.dim_model = dim_model
        self.num_head = num_head
        self.dim_ff = dim_ff
        self.num_layer = num_layer

        self.decoder_block = [Decoder_layer(self.dim_model,self.num_head,self.dim_ff) for _ in range(self.num_layer)]

    def __call__(self, tar, out_enc, pad_mask, look_ahead_mask):
        for i in range(self.num_layer):
            out = self.decoder_block[i](tar,out_enc, pad_mask, look_ahead_mask)
        return out

## Transformer

In [15]:
class Transformer(tf.keras.Model):
    #Transforemr에서 사용하는 Parameter는 여러군데에서 사용이 되어서 혼동이 될 수 있는데 아래를 참고하자.
    #dim_model : dim_model 자체는 Multi-head attention에서 사용을 하는 dimension인데, input = output을 유지하기 위해서, dim_embedding, depth*num_head와 동일한 값을 가진다.
    #num_head : multi-head에서 몇 개의 head로 병렬 연산을 할 것인지 결정하는 것이다.
    #dim_ff : ffnn의 Dense layer에서 사용되는 dimension으로 특정되는 숫자가 있지는 않다.
    #num_layer : encoder와 decoder layer들의 반복 횟수이다.
    #max_token : vectorize할 때 최대로 사용할 token의 숫자이고, embedding할 때도 동일하게 가져간다.
    #seq_len : 한 문장 안에서 input으로 처리할 최대의 단어 숫자를 의미
    def __init__(self, dim_model, num_head, dim_ff, num_layer, input_vocab_size, output_vocab_size, seq_len):
        super(Transformer, self).__init__()
        self.dim_model = dim_model
        self.num_head = num_head
        self.dim_ff = dim_ff
        self.num_layer = num_layer
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.seq_len = seq_len

        self.encoder = Encoder(self.dim_model, self.num_head, self.dim_ff, self.num_layer)
        self.decoder = Decoder(self.dim_model, self.num_head, self.dim_ff, self.num_layer)
        # self.vectorizer_src = tf.keras.layers.TextVectorization(max_tokens=self.max_token,output_mode='int',output_sequence_length=self.seq_len)
        # self.vectorizer_tar = tf.keras.layers.TextVectorization(max_tokens=self.max_token,output_mode='int',output_sequence_length=self.seq_len)
        self.embedding_src = tf.keras.layers.Embedding(input_dim=self.input_vocab_size, output_dim=self.dim_model)
        self.embedding_tar = tf.keras.layers.Embedding(input_dim=self.output_vocab_size, output_dim=self.dim_model)
        self.final_layer = tf.keras.layers.Dense(self.output_vocab_size)

    def __call__(self, src_lang, tar_lang):
        #src_lang, tar_lang은 vectorized된 input을 받는게 좋다. 매번 실행할 때마다, vocab을 학습하고, vectorize를 하는 것은 비효율적이기 때문임
        #따라서 input의 형태는 (batch_size, seq_length)이다.

        
        # #src_lang vectorize
        # src_slices = tf.data.Dataset.from_tensor_slices(src_lang)
        # self.vectorizer_src.adapt(src_slices)
        # src_lang = self.vectorizer_src(src_lang)

        #vectorized된 input으로 pad_mask 생성
        pad_mask = create_pad_mask(src_lang)

        #vectorized된 input으로 embedding + positional encoding 실행
        src_embedding = self.embedding_src(src_lang)
        src_positionalencoding = positional_encoding(src_embedding)
        src_embedded = src_embedding * tf.math.sqrt(tf.cast(self.dim_model, tf.float32)) + src_positionalencoding

        #encoder 실행
        out_enc = self.encoder(src_embedded, pad_mask)

        # #tar_lang vectorize
        # tar_slices = tf.data.Dataset.from_tensor_slices(tar_lang)
        # self.vectorizer_tar.adapt(tar_slices)
        # tar_lang = self.vectorizer_tar(tar_lang)

        #vectorized된 input으로 look ahead mask 생성
        look_ahead_mask = create_mask(tar_lang)

        #vectorized된 input으로 embedding + positional encoding 실행
        tar_embedding = self.embedding_tar(tar_lang)
        tar_positionalencoding = positional_encoding(tar_embedding)
        tar_embedded = tar_embedding * tf.math.sqrt(tf.cast(self.dim_model, tf.float32)) + tar_positionalencoding

        #decoder 실행
        out_dec = self.decoder(tar_embedded, out_enc, pad_mask, look_ahead_mask)

        #generator 실행
        out = self.final_layer(out_dec)
        
        return out        

## 하이퍼파라미터 설정

In [16]:
dim_model = 256
num_head = 8
dim_ff = 1028
num_layer = 3
max_token = 20000
seq_len = 40

# input_vocab_size = vocab_size_ko = vectorizer_ko.vocab_size + 2
# output_vocab_size = vocab_size_en = vectorizer_en.vocab_size + 2

## loss function

In [17]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #from_logits=True로 하면 Dense 이후 softmax layer 값 출력

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) # 예를 들어서 실제 자료(0은 패딩)가 [1,2,3,4,5,0,0,0,0,0] 이라면 [0,0,0,0,0,1,1,1,1,1]로 바꿔 줌
                                                     # 이후 tf.math.logical_not을 활용해서 [True,True,True,True,True,False,False,False,False,False]으로 바꿔 줌
  loss_ = loss_object(real, pred) # loss_는 패딩을 고려하지 않은 loss 값

  mask = tf.cast(mask, dtype=loss_.dtype) # [True,True,True,True,True,False,False,False,False,False]를 [1,1,1,1,1,0,0,0,0,0] 으로 바꿔 줌
  loss_ *= mask # loss에 mask를 곱해서, 패딩인 부분은 0처리 해줌

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## optimizer

In [18]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, dim_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.dim_model = dim_model
    self.dim_model = tf.cast(self.dim_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    step = tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.dim_model) * tf.math.minimum(arg1, arg2)

## 데이터 로드

In [19]:
import pandas as pd

# 엑셀 파일 경로
file_path = 'kor_enc.xlsx'

# 엑셀 파일 로드
df = pd.read_excel(file_path)

# 데이터 확인
print(df.head())

   SID                                                 원문  \
0    1  'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1    2                                       씨티은행에서 일하세요?   
2    3              푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   
3    4   11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.   
4    5     6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.   

                                                 번역문  
0  Bible Coloring' is a coloring application that...  
1                        Do you work at a City bank?  
2  PURITO's bestseller, which recorded 4th rough ...  
3  In Chapter 11 Jesus called Lazarus from the to...  
4  I would feel grateful to know how many stocks ...  


In [20]:
# 한국어 및 영어 데이터 추출
korean_texts = df['원문'].astype(str).tolist()
english_texts = df['번역문'].astype(str).tolist()

# 한국어 텍스트 벡터화
vectorizer_ko = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(korean_texts, target_vocab_size=max_token)

# 영어 텍스트 벡터화
vectorizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(english_texts, target_vocab_size=max_token)

import pickle as pkl

# 오래 걸리는 부분 저장하기
with open('vectorizers.pkl', 'wb') as f:
    pkl.dump([vectorizer_ko, vectorizer_en], f)

# 가져다 쓰기
with open('vectorizers.pkl', 'rb') as f:
    vectorizer_ko, vectorizer_en = pkl.load(f)

In [21]:
start_token_ko, end_token_ko = [vectorizer_ko.vocab_size], [vectorizer_ko.vocab_size + 1]
start_token_en, end_token_en = [vectorizer_en.vocab_size], [vectorizer_en.vocab_size + 1]
vocab_size_ko = vectorizer_ko.vocab_size + 2
vocab_size_en = vectorizer_en.vocab_size + 2

In [22]:
print('시작 토큰 번호 :',start_token_ko, start_token_en)
print('종료 토큰 번호 :',end_token_ko, end_token_en)
print('단어 집합의 크기 :',vocab_size_ko)
print('단어 집합의 크기 :',vocab_size_en)

시작 토큰 번호 : [19708] [20012]
종료 토큰 번호 : [19709] [20013]
단어 집합의 크기 : 19710
단어 집합의 크기 : 20014


In [23]:
# 최대 길이를 40으로 정의
seq_length = seq_len

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def vectorize_and_filter(src, tar):
  vectorized_src, vectorized_tar = [], []
  
  for (sentence_ko, sentence_en) in zip(src, tar):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence_ko = start_token_ko + vectorizer_ko.encode(sentence_ko) + end_token_ko
    sentence_en = start_token_en + vectorizer_en.encode(sentence_en) + end_token_en
      
    if len(sentence_ko) <= seq_length and len(sentence_en) <= seq_length:
      vectorized_src.append(sentence_ko)
      vectorized_tar.append(sentence_en)
  
  # 패딩
  vectorized_src = tf.keras.preprocessing.sequence.pad_sequences(
      vectorized_src, maxlen=seq_length, padding='post')
  vectorized_tar = tf.keras.preprocessing.sequence.pad_sequences(
      vectorized_tar, maxlen=seq_length, padding='post')
  
  return vectorized_src, vectorized_tar

In [24]:
ko_text, en_text = vectorize_and_filter(korean_texts, english_texts)

In [25]:
#train, validation, test set 나누기
korean_train, korean_val, korean_test = ko_text[:180000], ko_text[180000:195000], ko_text[195000:]
english_train, english_val, english_test = en_text[:180000], en_text[180000:195000], en_text[195000:]

In [26]:
batch_size = 32
buffer_size = len(en_text)

dataset_train = tf.data.Dataset.from_tensor_slices((korean_train, english_train)).batch(batch_size).shuffle(buffer_size).prefetch(tf.data.experimental.AUTOTUNE)
dataset_val = tf.data.Dataset.from_tensor_slices((korean_val, english_val)).batch(batch_size).shuffle(buffer_size).prefetch(tf.data.experimental.AUTOTUNE)
dataset_test = tf.data.Dataset.from_tensor_slices((korean_test, english_test)).batch(batch_size)

In [27]:
transformer = Transformer(dim_model=dim_model,num_head=num_head,dim_ff=dim_ff,num_layer=num_layer,input_vocab_size=vocab_size_ko,output_vocab_size=vocab_size_en,seq_len=seq_len)

In [28]:
learning_rate = CustomSchedule(dim_model)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, seq_len - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

transformer.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [29]:
# 인풋, 아웃풋의 텐셔 shape 정의
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
]

# tf.function을 사용하면 그래프를 미리 컴파일 하기 때문에 속도가 상당히 빠름
# 같은 GPU여도 케라스에 비해서 체감상 7~8배 정도의 차이가 나는 것 같음
# @tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  with tf.GradientTape() as tape:
    predictions = transformer(inp, tar_inp)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [30]:
# 저장할 체크포인트 지정
checkpoint_path = "./"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


import time
# 5 에포크 훈련
for epoch in range(5):
  start = time.time()
  
  train_loss.reset_state()
  train_accuracy.reset_state()
  
  # input : 한국어, tar : 영어
  for (batch, (inp, tar)) in enumerate(dataset_train):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

In [33]:
class Translator():
    def __init__(self, transformer, vectorizer_ko, vectorizer_en):
        self.transformer = transformer
        self.vectorizer_ko = vectorizer_ko
        self.vectorizer_en = vectorizer_en

    def __call__(self, sentence, seq_length):
        self.start_en = self.vectorizer_en.vocab_size
        self.end_en = self.vectorizer_en.vocab_size + 1
        self.new_vocab = [vectorizer_en.decode([i]) for i in range(vectorizer_en.vocab_size)] + ['<start>', '<end>']
        self.new_vectorizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(self.new_vocab, target_vocab_size=vectorizer_en.vocab_size + 2)
        self.sentence = tf.expand_dims(self.vectorizer_ko.encode(sentence), axis=0)  # 차원 추가
        
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
        output_array = output_array.write(0, [self.start_en])

        for i in tf.range(seq_length):
            output = tf.transpose(output_array.stack())
            predictions = self.transformer(self.sentence, output)

            predictions = predictions[:, -1:, :]  
            predicted_id = tf.argmax(predictions, axis=-1)

            output_array = output_array.write(i + 1, predicted_id[0])

            if tf.reduce_all(predicted_id[0][0] == self.end_en):
                break

        output = tf.transpose(output_array.stack())
        final_output = self.new_vectorizer_en.decode(tf.squeeze(output))

        return final_output


In [34]:
translator = Translator(transformer, vectorizer_ko, vectorizer_en)

In [35]:
sentence = "나는 한국인이다."

In [36]:
translator(sentence, 40)

'�start℃yourselfz +�'